In [1]:
import kugupu as kgp
import MDAnalysis as mda
import nglview as nv
import numpy as np

_ColormakerRegistry()

In [2]:
res = kgp.load_results('datafiles/results.hdf5')

In [3]:
def add_names(u):
    # Guesses atom names based upon masses
    def approx_equal(x, y):
        return abs(x - y) < 0.1
    
    # mapping of atom mass to element
    massdict = {}
    for m in set(u.atoms.masses):
        for elem, elem_mass in mda.topology.tables.masses.items():
            if approx_equal(m, elem_mass):
                massdict[m] = elem
                break
        else:
            raise ValueError
            
    u.add_TopologyAttr('names')
    for m, e in massdict.items():
        u.atoms[u.atoms.masses == m].names = e

In [4]:
u = mda.Universe('./datafiles/C6.data', './datafiles/C6.dcd')

In [5]:
add_names(u)

In [6]:
# for each fragment, calculate links to all other fragments
# fragment is linked to the closest fragment

In [7]:
frags = u.atoms.fragments
frags = [frags[1], frags[2], frags[4], frags[5]]

In [8]:
kgp.visualise.draw_fragments(*frags)

/Users/richardgowers/miniconda3/envs/kgp/lib/python3.7/site-packages/MDAnalysis/coordinates/base.py:837: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


NGLWidget()

In [11]:
for f in frags:
    mda.lib.mdamath.make_whole(f)

In [12]:
pos = np.stack([f.center_of_geometry() for f in frags])

In [35]:
distmat = mda.lib.distances.distance_array(pos, pos, box=frags[0].dimensions)

In [16]:
distmat[np.diag_indices_from(distmat)] = np.inf

In [23]:
import networkx as nx

In [26]:
g = nx.Graph(distmat)

In [36]:
list(nx.minimum_spanning_edges(nx.Graph(distmat)))

[(0, 2, {'weight': 17.39614873932999}),
 (1, 2, {'weight': 31.19675425833221}),
 (1, 3, {'weight': 50.96351799858536})]

In [34]:
distmat

array([[        inf, 38.91848795, 17.39614874, 56.57529388],
       [        inf,         inf, 31.19675426, 50.963518  ],
       [        inf, 31.19675426,         inf, 54.68561074],
       [        inf, 50.963518  , 54.68561074,         inf]])

In [20]:
distmat[:, 0] = np.inf

In [21]:
distmat

array([[        inf, 38.91848795, 17.39614874, 56.57529388],
       [        inf,         inf, 31.19675426, 50.963518  ],
       [        inf, 31.19675426,         inf, 54.68561074],
       [        inf, 50.963518  , 54.68561074,         inf]])

In [ ]:
def draw_fragments(*frags):
    """Draw fragments with nglview
    
    Parameters
    ----------
    frags : list of AtomGroup
    """
    # isolate fragments into their own Universe
    
    # make sure all molecules are whole
    
    # make sure all in same image
    
    

In [16]:
mol = u.atoms.fragments[0]
mol2 = u.atoms.fragments[1]

In [57]:
from io import StringIO

f = mda.lib.util.NamedStream(StringIO(), 'tmp.pdb')


In [58]:
ag = mol + mol2
with mda.Writer(f, n_atoms=len(ag)) as w:
    w.write(ag)
f.seek(0)
u_new = mda.Universe(f)

In [62]:
for frag in u_new.atoms.fragments:
    mda.lib.mdamath.make_whole(frag)

In [63]:
v = nv.show_mdanalysis(u_new)
v

NGLWidget()

In [42]:
u.trajectory.dimensions_array

array([[82.370605, 82.370605, 82.370605, 90.      , 90.      , 90.      ],
       [82.50731 , 82.50731 , 82.50731 , 90.      , 90.      , 90.      ],
       [82.38418 , 82.38418 , 82.38418 , 90.      , 90.      , 90.      ],
       [82.480804, 82.480804, 82.480804, 90.      , 90.      , 90.      ],
       [82.402664, 82.402664, 82.402664, 90.      , 90.      , 90.      ]],
      dtype=float32)

In [49]:
mda.lib.mdamath.make_whole(mol)
mda.lib.mdamath.make_whole(mol2)

In [50]:
v = nv.NGLWidget()


v.add_trajectory(mol + mol2)
v

/Users/richardgowers/miniconda3/envs/kgp/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  "".format(attrname, default))
/Users/richardgowers/miniconda3/envs/kgp/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'resnames' Using default value of 'UNK'
  "".format(attrname, default))
/Users/richardgowers/miniconda3/envs/kgp/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  "".format(attrname, default))
/Users/richardgowers/miniconda3/envs/kgp/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  "".format(attrname, default))
/Users/richardgowers/miniconda3/envs/kgp/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found n

NGLWidget(max_frame=4)

In [15]:
v = nv.show_mdanalysis(mol)
#v.add_ball_and_stick()
v

NGLWidget(max_frame=4)

In [20]:
v.download_image??

In [21]:
v = nv.demo()
v

NGLWidget()